# Divergent data reduction for Amor

In this notebook, we will look at how to use the `essreflectometry` package with Sciline, for reflectometry data collected from the PSI instrument [Amor](https://www.psi.ch/en/sinq/amor) in [divergent beam mode](https://www.psi.ch/en/sinq/amor/selene).

We will begin by importing the modules that are necessary for this notebook.

## Setup

In [ ]:
import scipp as sc
from ess import amor
from ess.amor import data  # noqa: F401
from ess.reflectometry.types import *
from ess.amor.types import *

## Create and configure the workflow

We begin by creating the Amor workflow object which is a skeleton for reducing Amor data,
with pre-configured steps.

In [ ]:
workflow = amor.AmorWorkflow()

We then need to set the missing parameters which are specific to each experiment:

In [ ]:
workflow[SampleSize[SampleRun]] = sc.scalar(10.0, unit='mm')
workflow[SampleSize[ReferenceRun]] = sc.scalar(10.0, unit='mm')

workflow[ChopperPhase[ReferenceRun]] = sc.scalar(-7.5, unit='deg')
workflow[ChopperPhase[SampleRun]] = sc.scalar(-7.5, unit='deg')

workflow[QBins] = sc.geomspace(dim='Q', start=0.005, stop=0.3, num=391, unit='1/angstrom')
workflow[WavelengthBins] = sc.geomspace('wavelength', 2.8, 12, 301, unit='angstrom')

# The YIndexLimits and ZIndexLimits define ranges on the detector where
# data is considered to be valid signal.
# They represent the lower and upper boundaries of a range of pixel indices.
workflow[YIndexLimits] = sc.scalar(11, unit=None), sc.scalar(41, unit=None)
workflow[ZIndexLimits] = sc.scalar(80, unit=None), sc.scalar(370, unit=None)

In [ ]:
workflow.visualize(ReflectivityOverQ, graph_attr={'rankdir': 'LR'})

## Caching the reference result

The reference result (used for normalizing the sample data) only needs to be computed once.
It represents the intensity reflected by the super-mirror.

We compute it using the pipeline and thereafter set the result back on the original pipeline.

In [ ]:
workflow[Filename[ReferenceRun]] = amor.data.amor_reference_run()
# The sample rotation value in the file is slightly off, so we set it manually
workflow[SampleRotation[ReferenceRun]] = sc.scalar(0.65, unit='deg')

reference_result = workflow.compute(IdealReferenceIntensity)
# Set the result back onto the pipeline to cache it
workflow[IdealReferenceIntensity] = reference_result

If we now visualize the pipeline again, we can see that the reference is not re-computed:

In [ ]:
workflow.visualize(ReflectivityOverQ, graph_attr={'rankdir': 'LR'})

## Computing sample reflectivity

We now compute the sample reflectivity from 3 runs that used different sample rotation angles.
The different rotation angles cover different ranges in $Q$.

In [ ]:
runs = {
    '608': sc.scalar(0.85, unit='deg'),
    '609': sc.scalar(2.25, unit='deg'),
    '610': sc.scalar(3.65, unit='deg'),
    '611': sc.scalar(5.05, unit='deg'),
}

results = {}
for file, angle in runs.items():
    workflow[Filename[SampleRun]] = amor.data.amor_sample_run(file)
    workflow[SampleRotation[SampleRun]] = angle
    results[file] = workflow.compute(ReflectivityOverQ).hist()

sc.plot(results, norm='log', vmin=1e-4)

In [ ]:
from ess.reflectometry.tools import scale_reflectivity_curves_to_overlap
results_scaled = dict(zip(
    results.keys(),
    scale_reflectivity_curves_to_overlap(results.values())[0],
    strict=True
))
sc.plot(results_scaled, norm='log', vmin=1e-5)

In [ ]:
from ess.reflectometry.tools import combine_curves
c = combine_curves(results_scaled.values(), workflow.compute(QBins))
c.plot(norm='log')

### Additional diagnostics plots

In [ ]:
workflow[Filename[SampleRun]] = amor.data.amor_sample_run(608)
workflow[SampleRotation[SampleRun]] = sc.scalar(0.85, unit='deg')
workflow.compute(ReflectivityDiagnosticsView)

## Make a $(\lambda, \theta)$ map
A good sanity check is to create a two-dimensional map of the counts in $\lambda$ and $\theta$ bins and make sure the triangles converge at the origin.

In [ ]:
workflow.compute(WavelengthThetaFigure)

This plot can be used to check if the value of the sample rotation angle $\omega$ is correct. The bright triangles should be pointing back to the origin $\lambda = \theta = 0$. In the figure above the black lines are all passing through the origin.

## Save data

We can save the computed $I(Q)$ to an [ORSO](https://www.reflectometry.org) [.ort](https://github.com/reflectivity/file_format/blob/master/specification.md) file using the [orsopy](https://orsopy.readthedocs.io/en/latest/index.html) package.

First, we need to collect the metadata for that file.
To this end, we build a pipeline with additional providers.
We also insert a parameter to indicate the creator of the processed data.

In [ ]:
from ess.reflectometry import orso
from orsopy import fileio

In [ ]:
workflow[orso.OrsoCreator] = orso.OrsoCreator(
    fileio.base.Person(
        name='Max Mustermann',
        affiliation='European Spallation Source ERIC',
        contact='max.mustermann@ess.eu',
    )
)

In [ ]:
workflow.visualize(orso.OrsoIofQDataset, graph_attr={'rankdir': 'LR'})

We build our ORSO dataset from the computed $I(Q)$ and the ORSO metadata:

In [ ]:
iofq_dataset = workflow.compute(orso.OrsoIofQDataset)
iofq_dataset

We also add the URL of this notebook to make it easier to reproduce the data:

In [ ]:
iofq_dataset.info.reduction.script = (
    'https://scipp.github.io/essreflectometry/examples/amor.html'
)

To support tracking provenance, we also list the corrections that were done by the workflow and store them in the dataset:

In [ ]:
iofq_dataset.info.reduction.corrections = orso.find_corrections(
    workflow.get(orso.OrsoIofQDataset)
)

Finally, we can save the data to a file.
Note that `iofq_dataset` is an [orsopy.fileio.orso.OrsoDataset](https://orsopy.readthedocs.io/en/latest/orsopy.fileio.orso.html#orsopy.fileio.orso.OrsoDataset).

In [ ]:
iofq_dataset.save('amor_reduced_iofq.ort')

Look at the first 50 lines of the file to inspect the metadata:

In [ ]:
!head amor_reduced_iofq.ort -n50